# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [28]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [29]:
df = pd.read_csv('event_datafile_new.csv')

# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [30]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [31]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print("Error when create keyspace: ", e)

#### Set Keyspace

In [32]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print("Error when set keyspace: ", e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

##### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4

In this query, we will select columns **arist, song, length** and the filters will be **sessionId, itemInSession**, so the the key will be the same set as a composite key.

In [33]:
session.execute('DROP TABLE IF EXISTS song_info_by_session_item')
session.execute("""
CREATE TABLE IF NOT EXISTS song_info_by_session_item (
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY ((sessionId, itemInSession))
)
""")

In [34]:
 ## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = """
SELECT artist, song, length FROM song_info_by_session_item
WHERE sessionId = 338 AND itemInSession = 4
"""                    

In [35]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[df.columns.get_loc("sessionId")]),
                                int(line[df.columns.get_loc("itemInSession")]),
                                line[df.columns.get_loc("artist")],
                                line[df.columns.get_loc("song")],
                                float(line[df.columns.get_loc("length")])))

In [36]:
## Add in the SELECT statement to verify the data was entered into the table
session.execute(query1)._current_rows 

[Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)]

##### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In this query, we will select columns **artist, song, user first/last name** and the filters will be **userId, sessionId**, so the the key will be the same set as a composite key.

In [37]:
query2 = """
SELECT artist, song, firstName, lastName FROM song_user_by_userid_sessionid
WHERE userId = 10 AND sessionId = 182
"""

In [38]:
 ## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute('DROP TABLE IF EXISTS song_user_by_userid_sessionid')
session.execute("""
CREATE TABLE IF NOT EXISTS song_user_by_userid_sessionid (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
""")      

In [39]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
 ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_by_userid_sessionid (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
         ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[df.columns.get_loc("userId")]),
                                int(line[df.columns.get_loc("sessionId")]),
                                int(line[df.columns.get_loc("itemInSession")]),
                                line[df.columns.get_loc("artist")],
                                line[df.columns.get_loc("song")],
                                line[df.columns.get_loc("firstName")],
                                line[df.columns.get_loc("lastName")]))

In [40]:
 ## Add in the SELECT statement to verify the data was entered into the table
session.execute(query2)._current_rows 

[Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz'),
 Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz'),
 Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')]

##### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this query, we will select columns **first/last name of the user** and the filters will be **song**. Since the userId can be used as the unique identifier for each user name(i.e. First Name and Last Name), we use song and userId as the primary keys.

In [41]:
query3 = """
SELECT firstName, lastName FROM user_by_song
WHERE song = 'All Hands Against His Own'
"""

In [42]:
 ## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute('DROP TABLE IF EXISTS user_by_song')
session.execute("""
CREATE TABLE IF NOT EXISTS user_by_song (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (song, userId)
)
""")      

In [43]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
         ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[df.columns.get_loc("song")],
                                int(line[df.columns.get_loc("userId")]),
                                line[df.columns.get_loc("firstName")],
                                line[df.columns.get_loc("lastName")]))

In [44]:
# Add in the SELECT statement to verify the data was entered into the table
session.execute(query3)._current_rows 

[Row(firstname='Jacqueline', lastname='Lynch'),
 Row(firstname='Tegan', lastname='Levine'),
 Row(firstname='Sara', lastname='Johnson')]

### Drop the tables before closing out the sessions

In [45]:
session.execute('DROP TABLE IF EXISTS song_info_by_session_item')
session.execute('DROP TABLE IF EXISTS song_user_by_userid_sessionid')
session.execute('DROP TABLE IF EXISTS user_by_song')

### Close the session and cluster connection¶

In [46]:
session.shutdown()
cluster.shutdown()